# Model engineering

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.svm import SVR
import pickle


df = pd.read_csv('../data/processed/yield.csv')
df.head()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,avg_temp
0,Albania,Maize,1990,36613,1485.0,16.37
1,Albania,Potatoes,1990,66667,1485.0,16.37
2,Albania,Rice,1990,23333,1485.0,16.37
3,Albania,Sorghum,1990,12500,1485.0,16.37
4,Albania,Soybeans,1990,7000,1485.0,16.37


In [2]:
encoder = OneHotEncoder(sparse_output=False)
encoded = pd.DataFrame(encoder.fit_transform(
    df[['Item', 'Area']]), columns=encoder.get_feature_names_out())
df = df.drop(['Area', 'Item'], axis=1).join(encoded)
X = df.drop(['hg/ha_yield'], axis=1)
y = df['hg/ha_yield']

In [3]:
X.head()

,Year,average_rain_fall_mm_per_year,avg_temp,Item_Cassava,Item_Maize,Item_Plantains,Item_Potatoes,Item_Rice,Item_Sorghum,Item_Soybeans,...,Area_Tajikistan,Area_Thailand,Area_Tunisia,Area_Turkey,Area_Uganda,Area_Ukraine,Area_United Kingdom,Area_Uruguay,Area_Zambia,Area_Zimbabwe
0,1990,1485.0,16.37,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1990,1485.0,16.37,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1990,1485.0,16.37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1990,1485.0,16.37,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1990,1485.0,16.37,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
y.head()

0    36613
1    66667
2    23333
3    12500
4     7000
Name: hg/ha_yield, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train.shape

(8393, 111)

In [6]:
def accuracy_score(actual, estimate):
    mae = mean_absolute_error(actual, estimate)
    mape = mean_absolute_percentage_error(actual, estimate)
    rmse = root_mean_squared_error(actual, estimate)
    r2 = r2_score(actual, estimate)
    print(f' MAE: {mae:.3f}\n MAPE: {mape:.3f}\n' +
          f' RMSE: {rmse:.3f}\n R2: {r2:.3f}')

## Baseline model

### Purpose

In [7]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
pred = regressor.predict(X_test)
print('Score for LinearRegression')
accuracy_score(y_test, pred)

Score for LinearRegression
 MAE: 17372.502
 MAPE: 0.644
 RMSE: 24759.865
 R2: 0.786


**Baseline model**: defined baseline accuracy for the predictor using `LinearRegression` with default parameters.

---
## Model selection

In [8]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
pred = tree.predict(X_test)
print('Score for DecisionTreeRegressor with default parameters')
accuracy_score(y_test, pred)

Score for DecisionTreeRegressor with default parameters
 MAE: 2980.270
 MAPE: 0.092
 RMSE: 8604.370
 R2: 0.974


In [9]:
forest = RandomForestRegressor()
forest.fit(X_train, y_train)
pred = forest.predict(X_test)
print('Score for RandomForestRegressor with default parameters')
accuracy_score(y_test, pred)

Score for RandomForestRegressor with default parameters
 MAE: 3054.816
 MAPE: 0.089
 RMSE: 7015.182
 R2: 0.983


In [10]:
svm = SVR(kernel='linear', C=100)
svm.fit(X_train, y_train)
pred = svm.predict(X_test)
accuracy_score(y_test, pred)

 MAE: 16036.324
 MAPE: 0.437
 RMSE: 26642.250
 R2: 0.753


In [11]:
svm = SVR(kernel='poly', C=100, gamma='auto', degree=2)
svm.fit(X_train, y_train)
pred = svm.predict(X_test)
accuracy_score(y_test, pred)

 MAE: 37075.998
 MAPE: 1.312
 RMSE: 57728.489
 R2: -0.161


After considering the performance difference between 4 different ML techniques, namely `DecisionTreeRegressor`, `RandomForestRegressor`, `SVR` with linear and polynomial kernels, the model selected is **Random Forest** for its great applicability and higher efficiency in learning than SVR.  
It obtains better score than the baseline model (`LinearRegression`) even without any adjustment. It is also better than `SVR` (and model computation takes less time).  
Also `SVR` might have problems related to having many encoded categorical values.

---
## Hyperparameter Tuning

In [ ]:
forest = RandomForestRegressor()
param_grid = {
    'min_samples_split': [2, 4, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['squared_error', 'friedman_mse', 'poisson'],
    'max_features': [None, 1, 'sqrt', 'log2']
}
grid = GridSearchCV(forest, param_grid, verbose=1, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_params_

Fitting 5 folds for each of 108 candidates, totalling 540 fits


{'criterion': 'poisson',
 'max_features': None,
 'min_samples_leaf': 2,
 'min_samples_split': 2}

Grid search gives us the best parameters for `RandomForestRegressor`:
- `n_estimators`: **50**
- `min_samples_split`: **4**
- `min_samples_leaf`: **2**
- `max_features`: **None**
- `criterion`: **poisson**

---
## Model evaulation

### Metrics

In [13]:
pred = grid.predict(X_test)
accuracy_score(y_test, pred)

 MAE: 3492.545
 MAPE: 0.093
 RMSE: 8951.360
 R2: 0.972


### Cross fold validation

In [14]:
score = cross_val_score(forest, X_train, y_train)
print(f'CV-score: {score.mean():.4f}')

CV-score: 0.9695


The model chosen is a `RandomForestRegressor`.  
- **Metrics**: with the parameters obtained from the previous optimization, the regressor gains only a slight increase in accuracy for much longer train time.
- **Cross validation**: performed cross validation with different number of estimators to see if it could help reduce the prediction error. A lower value could be used to limit resources usage as there is no significant increase in the CV-score.

In [ ]:
base = RandomForestRegressor()
base.fit(X_train, y_train)
base_pred = base.predict(X_test)
print('Score for base predictor')
accuracy_score(y_test, base_pred)
forest = grid.best_estimator_
forest.fit(X_train, y_train)
pred = forest.predict(X_test)
print('Score for best predictor')
accuracy_score(y_test, pred)

Score for base predictor
 MAE: 3027.912
 MAPE: 0.094
 RMSE: 8452.425
 R2: 0.975
Score for best predictor
 MAE: 3498.881
 MAPE: 0.093
 RMSE: 8995.100
 R2: 0.972


Since the fault parameters for `RandomForestRegressor` actually produce more complex (and overall more) trees, we are satisfied that there is only minimal increase in error.

---

## Feature importance and model interpretability

In [16]:
important = forest.feature_importances_.argsort()[::-1]
X.columns[important]

Index(['Item_Potatoes', 'Item_Cassava', 'Item_Sweet potatoes',
       'average_rain_fall_mm_per_year', 'Item_Yams', 'avg_temp',
       'Item_Plantains', 'Item_Rice', 'Area_Mexico', 'Area_India',
       ...
       'Area_Austria', 'Area_Malaysia', 'Area_Estonia', 'Area_Belarus',
       'Area_Netherlands', 'Area_Egypt', 'Area_Denmark', 'Area_Norway',
       'Area_Latvia', 'Area_Botswana'],
      dtype='object', length=111)

- **Feature importance**: from the model we can see above the most important features.
- Since the model has many rows due to the number of categories for the column `Area` (before encoding) Gini importance may not be accurate enough.

---

## Export the best model

In [17]:
with open('../artifacts/model.pkl', 'wb') as file:
    pickle.dump(grid.best_estimator_, file)
with open('../artifacts/encoder.pkl', 'wb') as file:
    pickle.dump(encoder, file)
with open('../artifacts/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

- **Model**: exported to `artifacts/model.pkl`
- **Encoder**: exported to `artifacts/encoder.pkl`
- **Scaler**: exported to `artifacts/scaler.pkl`

---

## Conclusions

- **Baseline model**: defined a baseline model using `LogisticRegression` with default parameters.
- **Model Selection**: inspected other models (`RandomForestRegressor` and `SVR`). Chosen `RandomForestRegressor`
- **Hyperparameter tuning**: best parameters for the regressor while also optimizing the model size.
- **Model evaluation**: measured accuracy of the model with **default metrics** and **cross validation**.
- **Feature importance**: displayed feature importance for model interpretability.
- **Export**: exported the model along with the scaler and encoder.